In [1]:
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.cm as cm  # colormaps

%matplotlib inline

In [2]:
import tensorflow as tf

# Extracting weights and gradients

Aims of this exercise: 

* to get and set model weights

* to inspect the weight-gradients for a single training example

* to inspect the unit activations for a single training example

* to examine the effect of rescaling the initial weights on gradients and on activations, for different activation functions 

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.initializers import TruncatedNormal, RandomNormal


Using TensorFlow backend.


In [4]:
layer_width = 20 # keep this large enough to allow statistical averaging, 
# and small enough to keep calculations feasible

the_activation = 'tanh'


# 10 layers of layer_width neurons

model = Sequential()
model.add(Dense(layer_width, input_dim=layer_width,  activation='relu'))
model.add(Dense(layer_width,  activation=the_activation, kernel_initializer=RandomNormal(mean=0.0,stddev=200.0)))
model.add(Dense(layer_width,  activation=the_activation, kernel_initializer=RandomNormal(mean=0.0,stddev=200.0)))
model.add(Dense(layer_width,  activation=the_activation, kernel_initializer=RandomNormal(mean=0.0,stddev=200.0)))
model.add(Dense(layer_width,  activation=the_activation, kernel_initializer=RandomNormal(mean=0.0,stddev=200.0)))
model.add(Dense(layer_width,  activation=the_activation, kernel_initializer=RandomNormal(mean=0.0,stddev=200.0)))
model.add(Dense(layer_width,  activation=the_activation, kernel_initializer=RandomNormal(mean=0.0,stddev=200.0)))
model.add(Dense(layer_width,  activation=the_activation, kernel_initializer=RandomNormal(mean=0.0,stddev=200.0)))
model.add(Dense(layer_width,  activation=the_activation, kernel_initializer=RandomNormal(mean=0.0,stddev=200.0)))
model.add(Dense(layer_width,  activation=the_activation, kernel_initializer=RandomNormal(mean=0.0,stddev=200.0)))
model.add(Dense(1)) # no activation - just linear output

# It appears that neither resetting the weights nor changing the initializer will affect what Keras does in setting weights
# 

We can get all the weights with which the model has been initialised, using a Keras function: 

In [5]:
model_weights = model.get_weights()

# as always, let's look at what we get. 
model_weights

# aha - a list of numpy arrays of weights

# look at the list : it is alternately neuron-to-neuron weights, and bias weights, which are initialised to 0 

[array([[ 0.26866317, -0.09731555, -0.27876139,  0.2817843 ,  0.28689361,
          0.23921943, -0.07260513,  0.24811465, -0.10941344, -0.37278181,
          0.09962404,  0.03434047,  0.27799404, -0.34355122, -0.08252171,
          0.21829963,  0.18818444,  0.01021168, -0.04272383, -0.3354407 ],
        [-0.20792618,  0.04835016, -0.2677685 , -0.26222029,  0.13665664,
         -0.14405291,  0.01320893, -0.04021472,  0.35372579,  0.04541311,
          0.23616433,  0.04605073,  0.09524089, -0.01568806,  0.35347676,
          0.02370307, -0.19806206, -0.21919461,  0.3215791 ,  0.0109854 ],
        [ 0.17214727,  0.33168793, -0.19773462,  0.11557102, -0.32894778,
         -0.0825654 , -0.04363918, -0.00226074,  0.33312619, -0.13782482,
         -0.20830441,  0.32584381, -0.15423699,  0.21733445, -0.08682916,
          0.11564153,  0.04380614,  0.13222009,  0.17697483, -0.20503892],
        [-0.09908625,  0.34742981, -0.13408574, -0.31114537,  0.01972511,
          0.16123313, -0.08408761, 

We can multiply all weights by a constant factor, and then put them back into the model. 

*** In this way, we will be able to see the effect of initialising weights with different values: this will affect both gradients and activations throughout the model. ***

In [6]:
model.compile(loss='mse', optimizer='adam')

In [7]:
np.mean( np.abs( model.get_weights()[0]))

0.18888782

In [8]:
outputTensor = model.output #Or model.layers[index].output
outputTensor

<tf.Tensor 'dense_11/BiasAdd:0' shape=(?, 1) dtype=float32>

In [9]:
type(outputTensor)

tensorflow.python.framework.ops.Tensor

In [10]:
outputTensor.get_shape()

TensorShape([Dimension(None), Dimension(1)])

In [11]:
 listOfVariableTensors = model.trainable_weights

In [12]:
listOfVariableTensors

In [13]:
len(listOfVariableTensors)

22

This seems reasonable: there are 11 layers, and each layer will have input weights and a bias

We want to extract the list of 100x100 dense weights: we are interested in their gradients. 

In [14]:
# how do we extract alternate elements of a list? 
# Does the Python range function help? 

[ x for x in range(0,20,2)]

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [16]:
listOfVariableTensors[0].get_shape()  # Note that we are using get_shape() here, which is a method
# of the Tensor class


TensorShape([Dimension(20), Dimension(20)])

In [ ]:
# so to get just the neuron-to-neuron weights, we want alternate tensors starting at index 0
# we are not so interested in the biases just now (though of course we could be)

# This will be a list of just the large weight tensor matrices

listOfDenseWeightTensors  = [ listOfVariableTensors[x] for x in range(0,20,2)]

### Now we are going to use Tensorflow magic. 
Tensorflow is a language with back-propagation built in !!
The `gradients` function takes two tensors (in the same computation graph), and 
K.gradients(A,B) computes the gradient of A with respect to B. 

Tensorflow automatically back-propagates from A back to B, to find the gradients. 

*** point to understand ***: the `gradients` function actually creates a program (that is, 
it extends the computational graph containing A and B) that will calculate the gradients: the numerical values of the gradients are not yet defined because we have not given the computational graph any input. 

In [ ]:
from keras import backend as K

# you may simply be able to import tensorflow as tf (which we know is the backend)
# and then use tf.gradients instead of K.gradients

gradients = K.gradients(outputTensor, listOfDenseWeightTensors)

In [ ]:
gradients

***Point to understand!!!*** these gradients are Tensorflow nodes: they are calculations that are planned but haven't been done yet. 

In [ ]:
# define a training example

# this is a 1xlayer_width numpy array of Gaussian random numbers. 
# (We are just testing this network)
trainingExample = np.random.randn(1,layer_width)
trainingExample

The next thing to do is to run the Tensorflow graph and with the `trainingExample` we have just defined as the input. 

We can then read out the gradients from the (extended) graph that we have defined. 

In [ ]:
# run the network in Tensorflow itself: 
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# evaluate the gra the calculation that 
evaluated_gradients = sess.run(gradients,feed_dict={model.input:trainingExample})

In [ ]:
evaluated_gradients

In [ ]:
# that last output is confusing: what is the structure of evaluated_gradients? 
# let's find out

type(evaluated_gradients)

In [ ]:
len(evaluated_gradients)

In [ ]:
evaluated_gradients[0].shape

Ah.  So evaluated_gradients is a list of layer_widthxlayer_width arrays containing the gradients of 
all th weights in all the dense layers

But 10 large square arrays is too much information for our poor biological brains to process at once. 
We need to visualise it and summarise it. 

***What do we do? PLOT A SUMMARY ***

In [ ]:
mean_abs_gradients = [ np.mean(np.abs(x)) for x in evaluated_gradients ]

plt.plot(mean_abs_gradients)

## Activations

We will use a similar technique to extract the neural activations for each layer, for a random input. 

In [ ]:

def get_activations(model, layer, X_batch):
    # the next line defines a function in tensorflow for computing the activations of a particular layer
    # (indexed by the parameter `layer`) using the model, from an input
    get_activations = K.function([model.layers[0].input, K.learning_phase()], [model.layers[layer].output,])
    # this now applies the tensorflow function to get the activations of a particular layer
    activations = get_activations([X_batch,0])
    return activations

In [ ]:
last_activation = get_activations( model, 10, trainingExample)
last_activation

In [ ]:
all_activations = [ get_activations( model, x, trainingExample) for x in range(0,10) ] 

In [ ]:
type(all_activations)

In [ ]:
all_activations

In [ ]:
mean_abs_activations = [ np.mean(np.abs(x)) for x in all_activations]

plt.plot( mean_abs_activations )